In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from sklearn.model_selection import train_test_split
from functools import *
import sys
import os

PROJECT_ROOT = os.path.abspath(os.getcwd() + os.sep + os.pardir)
sys.path.insert(0, PROJECT_ROOT)

from lib.time_series_datasets import *
from lib.models import *
from lib.benchmarks import *
import notify2  # TODO replace notify watch here : https://notify2.readthedocs.io/en/latest/
from tensorflow import keras

DATA_ROOT = os.path.join(PROJECT_ROOT, "datasets")
TUNER_ROOT = os.path.join(PROJECT_ROOT, "models")
BENCHMARKS_ROOT = os.path.join(PROJECT_ROOT, "plots", "benchmarks")
WEIGHTS_ROOT = os.path.join(PROJECT_ROOT, "plots", "weights")
TB_ROOT = os.path.join(os.path.abspath(os.sep), "tmp", "tensorboard")

BENCHMARKS_DIR = "benchmarks"
WEIGHTS_DIR = "weights"

print("       Data dir:", DATA_ROOT)
print("      Tuner dir:", TUNER_ROOT)
print("Tensorboard dir:", TB_ROOT)

SKIP = False  # Skip all if a model is already tested?
OVERWRITE = False  # Redoing the model selection for a model?
BENCHMARKS_TRIALS = 10  # How many times do the benchmark. 0 to skip bechmark

TUNER = "BayesianOptimization"  # "Hyperband" or "BayesianOptimization"

MAX_UNITS = 400
MAX_EPOCHS = 150
PATIENCE = 10 # EarlyStopping
TRIALS = 3  # int positive number of iteration for one set of hyperparameter

MAX_TRIALS = 100  # BayesianOptimization

#set the following values based on the specific dataset
READOUT_ACTIVATION = keras.activations.softmax  # https://www.tensorflow.org/api_docs/python/tf/keras/activations  'softmax'
LOSS_FUNCTION = keras.losses.SparseCategoricalCrossentropy()  # https://www.tensorflow.org/api_docs/python/tf/keras/losses  'sparse_categorical_crossentropy'

if not os.path.exists(TUNER_ROOT):
    os.makedirs(TUNER_ROOT)
if not os.path.exists(TB_ROOT):
    os.makedirs(TB_ROOT)

benchmarks = BenchmarksDB(load_path=os.path.join(BENCHMARKS_ROOT, "benchmarks.json"), plot_path=WEIGHTS_ROOT)

       Data dir: /dati/luca/Uni-Luca/Tesi/progetto/datasets
      Tuner dir: /dati/luca/Uni-Luca/Tesi/progetto/models
Tensorboard dir: /tmp/tensorboard



|        | ArticularyWordRecognition | CharacterTrajectories | Epilepsy | JapaneseVowels  | Libras | SpokenArabicDigits |
|--------|:-------------------------:|:---------------------:|:--------:|:---------------:|:------:|:------------------:|
| Input  |             9             |           3           |    3     |       12        |   2    |         13         |
| Output |            25             |          20           |    4     |        9        |   15   |         10         |

In [11]:
from keras_tuner import Hyperband, BayesianOptimization


def model_selection(build_model_fn, names,
                    train_set, val_set,
                    tuner_path, verbose=1):
    dataset_name, class_name, experiment_name, model_name = names
    x_train, y_train = train_set
    x_val, y_val = val_set
    if TUNER == "Hyperband":
        working_dir = os.path.join(tuner_path, "Hyperband", dataset_name, class_name)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)

        tuner = Hyperband(
            build_model_fn,
            objective='val_accuracy',
            max_epochs=MAX_EPOCHS,
            hyperband_iterations=1.,
            seed=42,

            directory=working_dir,
            project_name=experiment_name + ' ' + model_name,
            overwrite=OVERWRITE,
            executions_per_trial=TRIALS,
        )
    elif TUNER == "BayesianOptimization":
        working_dir = os.path.join(tuner_path, "BayesianOptimization", dataset_name, class_name)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)

        tuner = BayesianOptimization(
            build_model_fn,
            objective='val_accuracy',
            max_trials=MAX_TRIALS,
            #num_initial_points=2,
            seed=42,

            directory=working_dir,
            project_name=experiment_name + ' ' + model_name,
            overwrite=OVERWRITE,
            executions_per_trial=TRIALS,
        )
    else:
        raise ValueError("Unknown Tuner -> {}".format(TUNER))

    # choose the best hyperparameters
    tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
                 callbacks=[
                     keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE)
                 ], verbose=verbose)
    return tuner

def testing_model(names, tuner, test_set,
                  tensorboard_path=None, benchmarks_verbose=0):
    dataset_name, class_name, experiment_name, model_name = names
    x_test, y_test = test_set  # keras.callbacks.CallbackList([])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)]
    if tensorboard_path is not None:
        tensorboard_dir = tensorboard_path + model_name
        callbacks.append(keras.callbacks.TensorBoard(tensorboard_dir, profile_batch='500,500'))

    print("Start {} benchmarks for {} | {} | {} | {}:".format(BENCHMARKS_TRIALS, dataset_name, class_name, experiment_name,
                                                              model_name))

    best_model_hp = tuner.get_best_hyperparameters()[0]
    test_model = None
    best_acc = 0
    best_tested = None
    metrics_ts = []
    loss_ts = []
    required_time = []

    tf.random.set_seed(42)

    for i in range(BENCHMARKS_TRIALS):
        initial_time = time()

        test_model = tuner.hypermodel.build(best_model_hp)
        test_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=MAX_EPOCHS,
                       callbacks=callbacks, verbose=benchmarks_verbose)
        loss, metrics = test_model.evaluate(x_train, y_train)
        if metrics > best_acc:
            best_acc = metrics
            best_tested = test_model
        required_time.append(time() - initial_time)
        metrics_ts.append(metrics)
        loss_ts.append(loss)

    stat = Statistic(best_model_hp, metrics_ts, loss_ts, required_time)

    return best_tested, stat


notify2_init = False


def send_notification(title, message):
    def notify2_init_fun():
        global notify2_init
        if not notify2_init:
            notify2.init("Tesi")

    notify2_init_fun()

    notice = notify2.Notification(title, message)
    notice.show()


# Local Hyperparameters

In [12]:
# This cell is a little nightmare

class HP:
    FREE = 0
    FIXED = 1
    RESTRICTED = 2

    def __init__(self, hp_type, value=None):
        self.value = value
        self.type = hp_type

    @classmethod
    def free(cls):
        return cls(HP.FREE)

    @classmethod
    def fixed(cls, value):
        return cls(HP.FIXED, value)

    @classmethod
    def restricted(cls, value=None):
        return cls(HP.RESTRICTED, value)


def get_int(tuner, name, hp, min_value, max_value, step=1, sampling=None, pn=None, pv=None):
    if hp.type == HP.FREE or hp.type == HP.RESTRICTED:
        tmp = tuner.Int(name, min_value=min_value, max_value=max_value, step=step, sampling=sampling, parent_name=pn,
                        parent_values=pv)
    elif hp.type == HP.FIXED:
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp


def get_float(tuner, name, hp, min_value, max_value, step=None, sampling=None, pn=None, pv=None):
    if hp.type == HP.FREE or (hp.type == HP.RESTRICTED and hp.value is None):
        tmp = tuner.Float(name, min_value=min_value, max_value=max_value, step=step, sampling=sampling, parent_name=pn,
                          parent_values=pv)
    elif hp.type == HP.FIXED or (hp.type == HP.RESTRICTED and hp.value is not None):
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp


def get_bool(tuner, name, hp):
    if hp.type == HP.FREE or hp.type == HP.RESTRICTED:
        tmp = tuner.Boolean(name)
    elif hp.type == HP.FIXED:
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp


def get_float_vec(tuner, name, hp, length, min_value, max_value, step=None, sampling=None, pn=None, pv=None):
    if hp.type == HP.FREE:
        tmp = [tuner.Float(name + ' ' + str(i), min_value=min_value, max_value=max_value, sampling=sampling,
                           parent_name=pn, parent_values=pv, step=step)
               for i in range(length)]
    elif hp.type == HP.FIXED:
        tmp = [tuner.Fixed(name + ' ' + str(i), hp.value[i]) for i in range(length)]
    elif hp.type == HP.RESTRICTED:
        tmp2 = tuner.Float(name + ' 0', min_value=min_value, max_value=max_value, sampling=sampling, parent_name=pn,
                               parent_values=pv, step=step)
        tmp = [tmp2 for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return tmp

def get_connectivity_esn1(tuner, hp):
    if hp.type == HP.FREE:
        tmp = tuner.Float('connectivity 0', min_value=0.1, max_value=1., step=0.1)
    elif hp.type == HP.FIXED:
        raise ValueError("HP.FIXED not valid for connectivity")
    elif hp.type == HP.RESTRICTED:
        tmp = tuner.Fixed('connectivity 0', hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp


def get_connectivity_esn2(tuner, hp, length):
    if hp.type == HP.FREE:
        tmp = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1.) for i in range(length)]
    elif hp.type == HP.FIXED:
        raise ValueError("HP.FIXED not valid for connectivity")
    elif hp.type == HP.RESTRICTED:
        if hp.value is None:
            tmp2 = tuner.Float('connectivity 0', min_value=0., max_value=1.)
        else:
            tmp2 = tuner.Fixed('connectivity 0', hp.value)
        tmp = [tmp2 for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return tmp

def get_connectivity(tuner, hp, length):  # It is a squared matrix
    if hp.type == HP.FREE:
        conn_matrix = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1., step = 0.1) if i == j else
                        tuner.Float('connectivity ' + str(i) + '->' + str(j), min_value=0., max_value=1., step = 0.1)
                        for i in range(length)]
                       for j in range(length)]
    elif hp.type == HP.FIXED:
        diagonal, off_diagonal = hp.value
        off_diagonal = tuner.Fixed('connectivity X->Y', off_diagonal)
        conn_matrix = [[tuner.Fixed('connectivity ' + str(i), diagonal) if i == j else
                        off_diagonal
                        for i in range(length)]
                       for j in range(length)]
    elif hp.type == HP.RESTRICTED:
        if hp.value is None:
            connectivity = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1., step=0.1) for i in range(length)]
        else:
            tmp = tuner.Fixed('connectivity 0', hp.value)
            connectivity = [ tmp for _ in range(length)]
        intra_connectivity = tuner.Float('connectivity X->Y', min_value=0., max_value=1., step=0.1)
        conn_matrix = [[connectivity[i] if i == j else intra_connectivity for i in range(length)] for j in range(length)]
    else:
        raise ValueError("HP type not found")
    return conn_matrix

def get_minmax(tuner, hp, length):  # Is a matrix for ESN3 and ESN4
    max_value = 2.0
    if hp.type == HP.FREE:
        minmax_vec = [[0. if i == j else tuner.Float('minmax ' + str(i) + '->' + str(j), min_value=0.1, max_value=max_value, step = 0.1)
                   for i in range(length)]
                  for j in range(length)]
    elif hp.type == HP.RESTRICTED:
        minmax = tuner.Float('minmax', min_value=0.1, max_value=max_value, step = 0.1)
        minmax_vec = [[0. if i == j else minmax
                   for i in range(length)]
                  for j in range(length)]
    elif hp.type == HP.FIXED:
        minmax = hp.value
        minmax_vec = [[0. if i == j else minmax
                   for i in range(length)]
                  for j in range(length)]
    else:
        raise ValueError("HP type not found")
    return minmax_vec

def get_spectral_radius(tuner, hp, length):  # Is a vector
    max_value = 2.0
    if hp.type == HP.FREE:
        sr_vec = [tuner.Float('spectral radius ' + str(i), min_value=0.1, max_value=max_value, step = 0.1) for i in range(length)]
    elif hp.type == HP.RESTRICTED:
        spectral_radius = tuner.Float('spectral radius 0', min_value=0.1, max_value=max_value, step = 0.1)
        sr_vec = [spectral_radius for _ in range(length)]
    elif hp.type == HP.FIXED:
        spectral_radius = hp.value
        sr_vec = [spectral_radius for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return sr_vec

def get_gsr(tuner, gsr):
    global_sr = get_bool(tuner, 'use G.S.R.', gsr)
    if global_sr:
        global_sr = tuner.Float('G.S.R.', min_value=0.1, max_value=2., parent_name='use G.S.R.', parent_values=True, step = 0.1)
    else:
        global_sr = None #  tuner.Fixed('G.S.R.', False, parent_name='use G.S.R.', parent_values=False)

    return global_sr


# Build model functions

In [13]:
def build_ESN1(output, _reservoirs,  # Defined by dataset
               units, spectral_radius, _gsr, _off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,  # Defined by experiment
               tuner) -> ESN1:
    tmp_model = ESN1(units=get_int(tuner, 'units', units, 50, MAX_UNITS),
                     connectivity=get_connectivity_esn1(tuner, connectivity),
                     spectral_radius=get_float(tuner, 'spectral radius 0', spectral_radius, min_value=0.1, max_value=2., step = 0.1),
                     output_units=output,
                     readout_activation=READOUT_ACTIVATION,
                     input_scaling=get_float(tuner, 'input scaling 0', input_scaling, min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=get_float(tuner, 'bias scaling 0', bias_scaling, min_value=0.1, max_value=1.5, step=0.1),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., step=0.1),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return tmp_model


def build_ESN2(output, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, _off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> ESN2:
    tmp_model = ESN2(units=get_int(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity_esn2(tuner, connectivity, reservoirs),
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     output_units=output,
                     readout_activation=READOUT_ACTIVATION,
                     input_scaling=get_float_vec(tuner, 'input scaling', input_scaling, reservoirs, min_value=0.1,
                                                 max_value=1.5, step=0.1),
                     bias_scaling=get_float_vec(tuner, 'bias scaling', bias_scaling, reservoirs, min_value=0.1,
                                                max_value=1.5, step=0.1),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., step=0.1),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return tmp_model


def build_ESN3(output, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> ESN3:
    tmp_model = ESN3(units=get_int(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity(tuner, connectivity, reservoirs),
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     off_diagonal=get_minmax(tuner, off_diag, reservoirs),
                     output_units=output,
                     readout_activation=READOUT_ACTIVATION,
                     input_scaling=get_float_vec(tuner, 'input scaling', input_scaling, reservoirs, min_value=0.1,
                                                 max_value=1.5, step=0.1),
                     bias_scaling=get_float_vec(tuner, 'bias scaling', bias_scaling, reservoirs, min_value=0.1,
                                                max_value=1.5, step=0.1),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., step=0.1),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'],
    )
    return tmp_model


def build_ESN4(output, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> ESN4:
    partitions = [tuner.Float('partition ' + str(i), min_value=0.1, max_value=1.0, step=0.333) for i in range(reservoirs)]
    total = sum(partitions)
    # Normalize the partition vector now sum(partitions) == 1.
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))
    tmp_model = ESN4(units=get_int(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity(tuner, connectivity, reservoirs),
                     partitions=partitions,
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     off_diagonal=get_minmax(tuner, off_diag, reservoirs),
                     output_units=output,
                     readout_activation=READOUT_ACTIVATION,
                     input_scaling=get_float_vec(tuner, 'input scaling', input_scaling, reservoirs, min_value=0.1,
                                                 max_value=1.5, step=0.1),
                     bias_scaling=get_float_vec(tuner, 'bias scaling', bias_scaling, reservoirs, min_value=0.1,
                                                max_value=1.5, step=0.1),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., step=0.1),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),# keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy']
    )
    return tmp_model

# Experiments configs

In [14]:
config = {
    'Datasets': [
        # "ArticularyWordRecognition",
        "CharacterTrajectories",
        # "Libras",
        # "SpokenArabicDigits",
        # "Epilepsy",
        # "JapaneseVowels"
    ],
    'Classes': [
        # "Best Models",
        "Multiple S.R.",
        # "Single S.R."
    ],
    'Models': [
        build_ESN1,
        build_ESN2,
        build_ESN3,
        build_ESN4
    ],
    #                Units         | Spectral radius | G.S.R.    | off-diag      | Connectivity     | Input scaling  | Bias scaling   | leaky    | learning rate
    #                F             | F-R             | F         | F - R         | F - R            | F-R            | F-R            | F        | F
    'Best Models': {
        'Best':      (HP.free(),     HP.free(),      HP.free(),  HP.restricted(), HP.restricted(),   HP.free(),       HP.free(),       HP.free(), HP.free())
    },
    'Multiple S.R.': {  # Modello con N sub res ora ha 7 + N ( N diag sr + 1 off diag st, 1 g.s.r., 1 off-diag conn, 1 input, 1 bias, 1 leaky, 1 learning)
         'Units 50': (HP.fixed(50),  HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
         'Units 75': (HP.fixed(75),  HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 100': (HP.fixed(100), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 150': (HP.fixed(150), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 250': (HP.fixed(250), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
    },
    'Single S.R.': {
         'Units 50': (HP.fixed(50),  HP.restricted(), HP.free(), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
         'Units 75': (HP.fixed(75),  HP.restricted(), HP.free(), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 100': (HP.fixed(100), HP.restricted(), HP.free(), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 150': (HP.fixed(150), HP.restricted(), HP.free(), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 250': (HP.fixed(250), HP.restricted(), HP.free(), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
    }
}

# Run all

In [15]:
from IPython.display import display

datasets = config.get('Datasets')
classes = config.get('Classes')
models_fn = config.get('Models')

run_time = time()
for dataset in datasets:
    train_path = os.path.join(DATA_ROOT, dataset, dataset + '_TRAIN.ts')
    test_path = os.path.join(DATA_ROOT, dataset, dataset + '_TEST.ts')

    x_train_all, y_train_all = load_sktime_dataset(train_path)
    x_test, y_test = load_sktime_dataset(test_path)

    x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                                      test_size=0.33, random_state=42, stratify=y_train_all)

    train_set = (x_train, y_train)  # Todo is this cast necessary?
    val_set = (x_val, y_val)
    test_set = (x_test, y_test)

    features = x_train.shape[-1]
    output_units = len(np.unique(y_test))  # Dataset must have one of each features

    for class_name in classes:
        class_time = time()
        for experiment, params in config.get(class_name).items():
            for model_fn in models_fn:
                model_name = model_fn.__annotations__['return'].__name__
                already_tested = benchmarks.is_benchmarked(dataset, class_name, experiment, model_name)

                if already_tested and SKIP:
                    continue

                build_fn = partial(model_fn, output_units, features, *params)
                names = (dataset, class_name, experiment, model_name)
                display(names)

                tuner = model_selection(build_fn, names,
                                        train_set, val_set,
                                        tuner_path=TUNER_ROOT, verbose=1)

                if BENCHMARKS_TRIALS > 0:
                    model, stat = testing_model(names, tuner, test_set)
                    benchmarks.add(dataset, class_name, experiment, model_name, stat)
                    #model.plot(names, path=WEIGHTS_ROOT, show=False)

        # benchmarks.save()
        # send_notification(class_name + " done", "Requested time:" + str(time() - class_time))

print("Total learning time:" + str(time() - run_time))
send_notification("All Done", "Requested time:" + str(time() - run_time))

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


('CharacterTrajectories', 'Multiple S.R.', 'Units 50', 'ESN1')

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 50 ESN1/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 50 ESN1/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks for CharacterTrajectories | Multiple S.R. | Units 50 | ESN1:
30/30 [==============================] - 0s 403us/step - loss: 0.6629 - accuracy: 0.7836


('CharacterTrajectories', 'Multiple S.R.', 'Units 50', 'ESN2')

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 50 ESN2/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 50 ESN2/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks for CharacterTrajectories | Multiple S.R. | Units 50 | ESN2:
30/30 [==============================] - 0s 394us/step - loss: 0.2433 - accuracy: 0.9380


...

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN2/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN2/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks for CharacterTrajectories | Multiple S.R. | Units 250 | ESN2:
30/30 [==============================] - 0s 378us/step - loss: 0.0791 - accuracy: 0.9863


('CharacterTrajectories', 'Multiple S.R.', 'Units 250', 'ESN3')

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN3/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN3/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks for CharacterTrajectories | Multiple S.R. | Units 250 | ESN3:
30/30 [==============================] - 0s 510us/step - loss: 0.1861 - accuracy: 0.9443


('CharacterTrajectories', 'Multiple S.R.', 'Units 250', 'ESN4')

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN4/oracle.json
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/progetto/models/BayesianOptimization/CharacterTrajectories/Multiple S.R./Units 250 ESN4/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 10 benchmarks for CharacterTrajectories | Multiple S.R. | Units 250 | ESN4:
30/30 [==============================] - 0s 556us/step - loss: 0.0871 - accuracy: 0.9716
Total learning time:759.9274864196777


In [16]:
benchmarks.save()